# Add games to analytics directory as json

## Setup

In [ ]:
import os
from os import listdir
from os.path import isfile, join

## Add Single Game 

In [3]:
game_path = "test_game.slp"
os.system('node stats.js' + ' ' + game_path)

0

## Add multiple games


In [1]:
games_path = "../../Slippi"
game_files = [games_path + '/' + f for f in listdir(games_path) if isfile(join(games_path, f))]

In [2]:
# this took about an hour for 6000 games
for file_name in game_files:
    os.system('node stats.js' + ' ' + file_name)

# Import game analytics into Pandas

## Setup

In [124]:
import numpy as np
import pandas as pd
import json

## Single game

In [246]:
def load_single_game(game_file, output):
    
    with open(game_file) as f:
        d = json.load(f)
    
    with open("characters.json") as f:
        characters = json.load(f)
    
    with open("moves.json") as f:
        moves = json.load(f)
    
    with open("stages.json") as f:
        stages = json.load(f)
    
    
    df = pd.json_normalize(d, max_level=1)
    if d['metadata']:
        x = pd.DataFrame()
        x['game_length'] = df['stats.playableFrameCount'] / 60
        x['stage'] = df['settings.stageId'].astype(str).map(stages)
        
        player_one = df["settings.players"][0][0]
        player_two = df["settings.players"][0][1]
        x['player_one_display_name'] = player_one["displayName"]
        x['player_two_display_name'] = player_two["displayName"]
        x['player_one_connect_code'] = player_one["connectCode"]
        x['player_two_connect_code'] = player_two["connectCode"]
        x['player_one_user_id'] = player_one["userId"]
        x['player_two_user_id'] = player_two["userId"]
    
        x['player_one_character'] = characters[str(player_one["characterId"])]['name']
        x['player_two_character'] = characters[str(player_two["characterId"])]['name']
    
        # These won't work till default colors are added (prepended) to characters.json
        # x['player_one_character_color'] = characters[str(player_one["characterId"])]['colors'][player_one["characterColor"]]
        # x['player_two_character_color'] = characters[str(player_two["characterId"])]['colors'][player_two["characterColor"]]
    
        stock_lost_counter = [0, 0]
        stocks_lost = [stock['playerIndex'] for stock in df['stats.stocks'][0] if stock['endFrame']]
        for stock in stocks_lost:
            if stock == 0:
                stock_lost_counter[0] += 1
            else:
                stock_lost_counter[1] += 1
    
        x['player_one_lost_stocks'] = stock_lost_counter[0]
        x['player_two_lost_stocks'] = stock_lost_counter[1]
        if stock_lost_counter[0] == stock_lost_counter[1]:
            x['winner'] = None
        elif stock_lost_counter[0] < stock_lost_counter[1]:
            x['winner'] = 0
        else:
            x['winner'] = 1
        x['played_at'] = df['metadata.startAt']
        x['overall'] = df['stats.overall']
        x['action_counts'] = df['stats.actionCounts']
        x['game_complete'] = df['stats.gameComplete']
        
        output = pd.concat([output, x])
        # print(output.shape)
        return output
    else:
        print("something went wrong with: ", game_file)
        # print(output.shape)
        return output


output = load_single_game('game_analytics/Game_20230817T194800.json', pd.DataFrame())
output

,game_length,stage,player_one_display_name,player_two_display_name,player_one_connect_code,player_two_connect_code,player_one_user_id,player_two_user_id,player_one_character,player_two_character,player_one_lost_stocks,player_two_lost_stocks,winner,played_at,overall,action_counts,game_complete
0,32.9,Pokémon Stadium,with,meow,WITH#0,MEOW#339,aPmT3TcxYLaUAlxSDKDsfKFmXC72,mwKYA5rvxaYBjYNpdEe8b39L7un2,Yoshi,Marth,0,0,None,2023-08-17T23:48:00Z,"[{'playerIndex': 0, 'inputCounts': {'buttons':...","[{'playerIndex': 0, 'wavedashCount': 0, 'wavel...",True


## Multiple games

In [247]:
analytics_path = "game_analytics"
analytics_files = [analytics_path + '/' + f for f in listdir(analytics_path) if isfile(join(analytics_path, f))]

output = pd.DataFrame()
for i, f in enumerate(analytics_files):
    # print(i)
    # if i>1000:
    #     break
    try:
        output = load_single_game(f, output)
    except:
        print("An exception occurred for: ") 
        print(f)


something went wrong with:  game_analytics/Game_20231002T112717.json
something went wrong with:  game_analytics/Game_20230821T074359.json
something went wrong with:  game_analytics/.DS_Store
something went wrong with:  game_analytics/Game_20231003T144755.json
something went wrong with:  game_analytics/Game_20230810T141105.json
something went wrong with:  game_analytics/Game_20231001T235749.json
something went wrong with:  game_analytics/Game_20231018T195855.json
something went wrong with:  game_analytics/Game_20230827T101530.json
something went wrong with:  game_analytics/Game_20230807T091818.json
something went wrong with:  game_analytics/Game_20230806T200651.json
something went wrong with:  game_analytics/Game_20230814T213419.json
something went wrong with:  game_analytics/Game_20230722T085734.json
something went wrong with:  game_analytics/Game_20231021T081930.json
something went wrong with:  game_analytics/Game_20230906T150537.json
something went wrong with:  game_analytics/Game_20

In [248]:
output



,game_length,stage,player_one_display_name,player_two_display_name,player_one_connect_code,player_two_connect_code,player_one_user_id,player_two_user_id,player_one_character,player_two_character,player_one_lost_stocks,player_two_lost_stocks,winner,played_at,overall,action_counts,game_complete
0,32.900000,Pokémon Stadium,with,meow,WITH#0,MEOW#339,aPmT3TcxYLaUAlxSDKDsfKFmXC72,mwKYA5rvxaYBjYNpdEe8b39L7un2,Yoshi,Marth,0,0,None,2023-08-17T23:48:00Z,"[{'playerIndex': 0, 'inputCounts': {'buttons':...","[{'playerIndex': 0, 'wavedashCount': 0, 'wavel...",True
0,217.083333,Yoshi's Story,with,VHSTAPE,WITH#0,VHS#396,aPmT3TcxYLaUAlxSDKDsfKFmXC72,WweIVWmBLXSa9Tv6VG8dmYPDJ8Z2,Yoshi,Peach,4,2,1,2023-08-09T23:13:15Z,"[{'playerIndex': 0, 'inputCounts': {'buttons':...","[{'playerIndex': 0, 'wavedashCount': 0, 'wavel...",True
0,102.533333,Pokémon Stadium,Matcha Man,with,MATC#818,WITH#0,Es93fbpkTRMBc1EPfiSHlfs2cMv1,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Peach,Yoshi,1,0,1,2023-09-11T20:02:15Z,"[{'playerIndex': 0, 'inputCounts': {'buttons':...","[{'playerIndex': 0, 'wavedashCount': 4, 'wavel...",True
0,46.583333,Yoshi's Story,with,Shlomp,WITH#0,CLOR#884,aPmT3TcxYLaUAlxSDKDsfKFmXC72,30cOiOMWLraUtgA8JltYkeO3cI82,Yoshi,Marth,0,0,None,2023-08-22T00:41:20Z,"[{'playerIndex': 0, 'inputCounts': {'buttons':...","[{'playerIndex': 0, 'wavedashCount': 0, 'wavel...",True
0,65.166667,Fountain of Dreams,ITSU,with,ITSU#117,WITH#0,NwRhZmeQYhQL13p8BCGqSLYGwAG2,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Dr. Mario,Yoshi,1,0,1,2023-07-24T04:17:04Z,"[{'playerIndex': 0, 'inputCounts': {'buttons':...","[{'playerIndex': 0, 'wavedashCount': 1, 'wavel...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,228.950000,Yoshi's Story,with,Jotyde,WITH#0,JOTY#425,aPmT3TcxYLaUAlxSDKDsfKFmXC72,OZhNnyBDstWxqxN8Sic9YPsns9B3,Yoshi,Fox,2,4,0,2023-09-14T03:33:20Z,"[{'playerIndex': 0, 'inputCounts': {'buttons':...","[{'playerIndex': 0, 'wavedashCount': 0, 'wavel...",True
0,36.716667,Pokémon Stadium,with,bradday0429,WITH#0,BRAD#102,aPmT3TcxYLaUAlxSDKDsfKFmXC72,sVNWjJMzpGQ4w4iZFxcp0nvnL3n2,Yoshi,Luigi,0,1,0,2023-10-22T05:43:25Z,"[{'playerIndex': 0, 'inputCounts': {'buttons':...","[{'playerIndex': 0, 'wavedashCount': 0, 'wavel...",True
0,157.400000,Fountain of Dreams,KEV,with,SSC#23,WITH#0,7eIllMJ4nCSpZnZwoYJCJMfKRnq2,aPmT3TcxYLaUAlxSDKDsfKFmXC72,Captain Falcon,Yoshi,4,2,1,2023-10-02T17:20:29Z,"[{'playerIndex': 0, 'inputCounts': {'buttons':...","[{'playerIndex': 0, 'wavedashCount': 0, 'wavel...",True
0,239.016667,Dream Land N64,with,ttocs,WITH#0,TTOX#608,aPmT3TcxYLaUAlxSDKDsfKFmXC72,W7JRcoIdRyMSCXHquOMoB8EOT7L2,Yoshi,Captain Falcon,4,2,1,2023-09-05T02:44:45Z,"[{'playerIndex': 0, 'inputCounts': {'buttons':...","[{'playerIndex': 0, 'wavedashCount': 0, 'wavel...",True
